# Kalman Filter
- Claude와 ESC를 학습을 위한 대화 중에 Kalman 필터에 관한 언급이 있었다.
- 이 기회이 칼만 필터에 관해서 조금 더 잘 알기 위해서 Claude에게 요청하여 아래 코드를 받았다.  

In [8]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 가상의 횡가속도 데이터 생성
def generate_lateral_acceleration(t):
    return 2 * np.sin(0.1 * t) + 0.5 * np.sin(0.4 * t)

# 노이즈 생성
def generate_noise(size, mean=0, std=0.5):
    return np.random.normal(mean, std, size)

# 칼만 필터 구현
class KalmanFilter:
    def __init__(self, initial_state, initial_estimate_error, measurement_noise, process_noise):
        self.state = initial_state
        self.estimate_error = initial_estimate_error
        self.measurement_noise = measurement_noise
        self.process_noise = process_noise

    def update(self, measurement):
        # 예측
        prediction = self.state
        prediction_error = self.estimate_error + self.process_noise

        # 업데이트
        kalman_gain = prediction_error / (prediction_error + self.measurement_noise)
        self.state = prediction + kalman_gain * (measurement - prediction)
        self.estimate_error = (1 - kalman_gain) * prediction_error

        return self.state

# 데이터 생성
t = np.linspace(0, 100, 1000)
true_acceleration = generate_lateral_acceleration(t)
noise = generate_noise(len(t))
noisy_acceleration = true_acceleration + noise

# 칼만 필터 적용
kf_true = KalmanFilter(0, 1, 0.1, 0.01)
kf_noisy = KalmanFilter(0, 1, 0.1, 0.01)

# measurement_noise와 process_noise를 모두 0으로 하면 칼만 필터가 동작하지 않음
# divison by zero 오류가 발생함
# kf_true = KalmanFilter(0, 1, 0, 0)
# kf_noisy = KalmanFilter(0, 1, 0, 0)


filtered_true = np.array([kf_true.update(x) for x in true_acceleration])
filtered_noisy = np.array([kf_noisy.update(x) for x in noisy_acceleration])

# Plotly를 사용한 시각화
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=("원본 데이터", "노이즈가 추가된 데이터"))

# 원본 데이터 그래프
fig.add_trace(go.Scatter(x=t, y=true_acceleration, mode='lines', name='실제 가속도'), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=filtered_true, mode='lines', name='필터링된 실제 가속도'), row=1, col=1)

# 노이즈가 추가된 데이터 그래프
fig.add_trace(go.Scatter(x=t, y=noisy_acceleration, mode='lines', name='노이즈 포함 가속도'), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=filtered_noisy, mode='lines', name='필터링된 노이즈 포함 가속도'), row=2, col=1)

fig.update_layout(height=800, width=1000, title_text="칼만 필터를 이용한 횡가속도 데이터 처리")
fig.show()

In [9]:
# true_acceleration, filtered_true, noisy_acceleration, filtered_noisy를 한 그래프에 그리기
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=noisy_acceleration, mode='lines', name='노이즈 포함 가속도'))
fig.add_trace(go.Scatter(x=t, y=filtered_noisy, mode='lines', name='필터링된 노이즈 포함 가속도'))
fig.add_trace(go.Scatter(x=t, y=true_acceleration, mode='lines', name='실제 가속도'))
fig.add_trace(go.Scatter(x=t, y=filtered_true, mode='lines', name='필터링된 실제 가속도'))
fig.update_layout(title_text="칼만 필터를 이용한 횡가속도 데이터 처리")
fig.show()


In [10]:
# filtered_noisy - filtered_true 그래프 그리기
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=filtered_noisy - filtered_true, mode='lines', name='필터링된 노이즈 포함 가속도 - 필터링된 실제 가속도'))
fig.update_layout(title_text="필터링된 노이즈 포함 가속도 - 필터링된 실제 가속도")
fig.show()


In [11]:
# (filtered_noisy - filtered_true) / filtered_true 그래프 그리기
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=(filtered_noisy - filtered_true) / filtered_true, mode='lines', name='(필터링된 노이즈 포함 가속도 - 필터링된 실제 가속도) / 필터링된 실제 가속도'))
fig.update_layout(title_text="(필터링된 노이즈 포함 가속도 - 필터링된 실제 가속도) / 필터링된 실제 가속도")

# y축 범위 설정
fig.update_yaxes(range=[-0.5, 0.5])
fig.show()


C:\Users\haese\AppData\Local\Temp\ipykernel_20508\3633603786.py:3: RuntimeWarning:

divide by zero encountered in divide

